In [16]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier



In [2]:
train = pd.read_csv('data_set_titanic/train.csv')
test = pd.read_csv('data_set_titanic/test.csv')

In [3]:
def populate_embarked(df):
    embarked = df.groupby(['Embarked']).count()['PassengerId']
    embarked_max = embarked[embarked == embarked.max()].index
    df.loc[df['Embarked'].isnull(), ['Embarked']] = embarked_max
    return df

In [4]:
mapping = {'Don': 'RoyaltyM', 'Mme': 'Miss', 'Ms': 'Miss', 'Major': 'Officer', 
           'Lady': 'RoyaltyF', 'Sir': 'RoyaltyM', 'Mlle': 'Miss', 'Col': 'Officer', 
           'Capt': 'Officer', 'Countness': 'RoyaltyF', 'Jonkheer': 'RoyaltyM', 'Dona': 'RoyaltyF'}

def set_titles(df, mapping):
    df['Title'] = df['Name'].str.extract(r'([A-Z]+[a-z]+)\.')
    df.replace({'Title': mapping}, inplace=True)
    return df

In [5]:
def populate_age(df):
    age_med = df.groupby('Title')['Age'].median()
    for title, age in age_med.iteritems():
        filter_age = df['Age'].isnull()
        filter_title = df['Title'] == title
        df.loc[filter_age & filter_title, 'Age'] = age
    return df

In [6]:
train = populate_embarked(train)
train = set_titles(train, mapping)
train = populate_age(train)
    
test = populate_embarked(test)
test = set_titles(test, mapping)
test = populate_age(test)

In [7]:
test['Fare'][test['Fare'].isnull()] = test['Fare'].median()

/home/leonid/PL_projects/LP/LP_ds/env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Замена Male / Female на 1 / 0 соответственно

In [8]:
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [9]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr


In [10]:
result = pd.DataFrame(test['PassengerId'])
test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [11]:
classes = {}

In [12]:
le = LabelEncoder()

In [13]:
le.fit(train['Sex'])
train['Sex'] = le.transform(train['Sex'])
classes['Sex'] = le.classes_

le.fit(train['Title'])
train['Title'] = le.transform(train['Title'])
classes['Title'] = le.classes_

le.fit(train['Embarked'])
train['Embarked'] = le.transform(train['Embarked'])
classes['Embarked'] = le.classes_

In [14]:
classes

{'Sex': array(['female', 'male'], dtype=object),
 'Title': array(['Countess', 'Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Officer', 'Rev',
        'RoyaltyF', 'RoyaltyM'], dtype=object),
 'Embarked': array(['C', 'Q', 'S'], dtype=object)}

In [15]:
le.fit(classes['Sex'])
test['Sex'] = le.transform(test['Sex'])

le.fit(classes['Title'])
test['Title'] = le.transform(test['Title'])

le.fit(classes['Embarked'])
test['Embarked'] = le.transform(test['Embarked'])

Вынос колонки Survived т.к. это целевой столбец

In [18]:
target = train['Survived']
train.drop(['Survived'], axis=1, inplace=True)

KeyError: 'Survived'

In [20]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

создание моделе предсказания

In [22]:
model_rf = RandomForestClassifier(n_estimators=100)
model_kn = KNeighborsClassifier(n_neighbors=20)

In [23]:
scores = cross_val_score(model_rf, train, target, cv=5)
print(scores)
print(scores.mean())

[0.78212291 0.78651685 0.85955056 0.75842697 0.82022472]
0.8013684012303056


In [24]:
scores = cross_val_score(model_kn, train, target, cv=5)
print(scores)
print(scores.mean())

[0.60335196 0.73595506 0.71348315 0.73033708 0.73595506]
0.7038164584771828


RandomFores работает лучше, поэтому будем обучать модель на RandomForest

In [25]:
model_rf = model_rf.fit(train, target)
predictions = model_rf.predict(test)

In [28]:
result.insert(1, 'Survived', predictions)

In [30]:
result.to_csv('data_set_titanic/result.csv')

In [32]:
!ls data_set_titanic

gender_submission.csv  result.csv  test.csv  train.csv
